### This notebook uses a breathing actuation sequence that computes triggers inflation deflation based upon an average inhale/exhale duration and a scale factor

In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import threading
import numpy as np
import time
from random import uniform

In [ ]:
def data_handler(unused_addr, args, data1, data2, data3, data4, data5, data6): #BITalino ServerBIT format
    global ecg_data, resp_data
    global osc_client
        
    B = args[0]
    HR = args[1]
    
    # BITalino ServerBIT
    ecg_data.append(float(np.random.rand(1)))
    resp_data.append(float(data6))
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])


In [ ]:
def riot_data_handler(unused_addr, args, *values): #R-IoT 
    global ecg_data, resp_data
    global osc_client
    
    B = args[0]
    HR = args[1]
    
#     ecg_data.append(float(values[13]))
    ecg_data.append(np.random.rand(1)[0])
    resp_data.append(float(values[12]))
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])
    


In [ ]:
def warmup(B, resp_data, HR, ecg_data):
    """Function to launch once the system has warmed up.
    Sets the  data initially and launches a recursive update of features.
    Parameters
    ----------
    B: object containing breathing data and features
    resp_data: respiration signal data to set
    HR: object containing ECG data and features
    ecg_data: ecg signal data to set
    """
    
    if not B.is_warmed_up:
        print("Breathing WARMUP")
        B.set_data(resp_data[-B.buffer_length:])
        B.is_warmed_up = True
        B.update_loop()
        
    if not HR.is_warmed_up:
        print("ECG WARMUP")
        HR.set_data(ecg_data[-HR.buffer_length:])
        HR.is_warmed_up = True
        HR.update_loop()
        
#         act_launcher(B, resp_data, HR, ecg_data) # No thread based

        timer_act = threading.Timer(0.1, act_launcher, [B, resp_data, HR, ecg_data] ) # Thread based
        timer_act.start() # Thread based
        
#     breathing_factor = B.breathing_factor
    
#     if B.routine == "None":
#         print("n-----")

#     elif B.routine == "breathing_biofeedback":
#         timer_actuation1 = threading.Timer(0.1, breathing_biofeedback, ["1", B, True] )
#         timer_actuation1.start()
    
#         timer_actuation2 = threading.Timer(0.1, breathing_biofeedback, ["2", B, True] )
#         #timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
#         timer_actuation2.start()

#     elif B.routine == "pulsating":
#         pass

In [ ]:
def act_launcher(B, resp_data, HR, ecg_data):
    global actuation_flag
    
    if B.routine == "None":
        print("n-------")

    elif B.routine == "breathing_biofeedback":
        breathing_factor = B.breathing_factor
        timer_actuation1 = threading.Timer(0.1, breathing_biofeedback, ["1", B, True, False, 1] )
        timer_actuation1.start()
    
        timer_actuation2 = threading.Timer(0.1, breathing_biofeedback, ["2", B, True, False, 1] )
        #timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
        timer_actuation2.start()

    elif B.routine == "pulsating":
        timer_actuation1 = threading.Timer(0.1, pulsating, ["1", True] )
        timer_actuation1.start()
    
        timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
        timer_actuation2.start()        

    elif B.routine == "random_inflation":
        timer_actuation1 = threading.Timer(0.1, random_inflation , ["1", 2, 5, True] )
        timer_actuation1.start()
    
        timer_actuation2 = threading.Timer(0.1, random_inflation, ["1", 2, 5, True] )
        timer_actuation2.start()        
    
    elif B.routine == "async_intervals":
        timer_actuation = threading.Timer(0.1, async_intervals, ["1", "2", 3, 3, True] )
        timer_actuation.start()

    elif B.routine == "async_breathing":
        timer_actuation = threading.Timer(0.1, async_breathing, ["1", "2", B, True, False, 1] )
        timer_actuation.start()    
    
    elif B.routine == "deflate3":
        timer_actuation1 = threading.Timer(0.1, deflate, ["1", 3] )
        timer_actuation1.start()
        timer_actuation1 = threading.Timer(0.1, deflate, ["2", 3] )
        timer_actuation1.start()
        
    elif B.routine == "inflate3":
        timer_actuation1 = threading.Timer(0.1, warmup_inflate, ["1", 3] )
        timer_actuation1.start()
        timer_actuation1 = threading.Timer(0.1, warmup_inflate, ["2", 3] )
        timer_actuation1.start()
        

In [ ]:
def warmup_inflate(act_id, duration):
    global osc_client
    
    osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


In [ ]:
def deflate(act_id, duration):
    global osc_client
    
    osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


### Different actuation sequences

In [ ]:
# computes inhale and exhale interval duration as average duration of breath * breathing_factor 
def breathing_biofeedback(act_id, B, inflate, hold, hold_time):
    global osc_client 
    global actuation_flag

  
    
    if not actuation_flag or B.routine != "breathing_biofeedback":
        return

#     if not actuation_flag:
#         return
    
    breathing_factor = B.breathing_factor
#     print("BrF: ", breathing_factor) 

    if hold and inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)
        timer_in = threading.Timer(hold_time, breathing_biofeedback, [act_id, B, True, False, hold_time])
        timer_in.start()
        
    if hold and not inflate: 
        osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)
        timer_in = threading.Timer(hold_time, breathing_biofeedback, [act_id, B, False, False, hold_time])
        timer_in.start()
    
    if not hold and inflate:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, False, True, hold_time])
        timer_exhale.start()
        
        for i in range(1,10):
            osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)

    if not hold and not inflate:
        print("exhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, True, True, hold_time])
        timer_inhale.start()
        
        for i in range(1,10):
            osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)


In [ ]:
# specify inhale and exhale interval duration in seconds
def breathing_intervals(act_id, inhale_duration, exhale_duration, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", inhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_exhale = threading.Timer(inhale_duration, breathing_intervals, [act_id, inhale_duration, exhale_duration, False])
        timer_exhale.start()

    else:
        print("exhale: ", exhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(exhale_duration, breathing_intervals, [act_id, inhale_duration, exhale_duration, True])
        timer_inhale.start()

In [ ]:
def pulsating(act_id,inflate):
    global osc_client
    global actuation_flag
    
    if not actuation_flag or B.routine != "pulsating":
        return
    
    if inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 50.0)
        timer_deflate = threading.Timer(3, pulsating, [act_id, False])
        timer_deflate.start()

    else:
        osc_client.send_message("/actuator/" + act_id + "/inflate", -50.0)
        timer_inflate = threading.Timer(3, pulsating, [act_id, True])
        timer_inflate.start()

In [ ]:
def random_inflation(act_id, lower, upper, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag or B.routine != "random_inflation":        
        return
    
    int_length = uniform(lower,upper)
    
    if inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_deflate = threading.Timer(int_length, random_inflation, [act_id, lower, upper, False])
        timer_deflate.start()

    else:
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inflate = threading.Timer(int_length, random_inflation, [act_id, lower, upper, True])
        timer_inflate.start()

In [ ]:
def async_intervals(act_first, act_snd, inhale_dur, exhale_dur, inflate_first):
    global osc_client, actuation_flag
    
    if not actuation_flag or B.routine != "async_intervals":
        return
    
    if inflate_first:
        print("inhale: ", inhale_dur)
        osc_client.send_message("/actuator/" + act_first + "/inflate", 100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", -100.0)
        timer_inhale = threading.Timer(inhale_dur, async_intervals, [act_first, act_snd, inhale_dur, exhale_dur, False])
        timer_inhale.start()

    else:
        print("exhale: ", exhale_dur)
        osc_client.send_message("/actuator/" + act_first + "/inflate", -100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 100.0)
        timer_exhale = threading.Timer(exhale_dur, async_intervals, [act_first, act_snd, inhale_dur, exhale_dur, True])
        timer_exhale.start()

In [ ]:
def async_breathing(act_first, act_snd, B, inflate, hold, hold_time):
    global osc_client, actuation_flag
    
    if not actuation_flag or B.routine != "async_breathing":
        return

    breathing_factor = B.breathing_factor
    
    if inflate and hold:
        osc_client.send_message("/actuator/" + act_first + "/inflate", 0.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 0.0)
        timer_in = threading.Timer(hold_time, async_breathing, [act_first, act_snd, B, True, False, hold_time])
        timer_in.start()
        
    if not inflate and hold:
        osc_client.send_message("/actuator/" + act_first + "/inflate", 0.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 0.0)
        timer_in = threading.Timer(hold_time, async_breathing, [act_first, act_snd, B, False, False, hold_time])
        timer_in.start()
        
    if inflate and not hold:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", 100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", -100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, False, True, hold_time])
        timer_exhale.start()

    if not inflate and not hold:
        print("exhale: ", B.features['avg_exhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", -100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 100.0)
        timer_inhale = threading.Timer(B.features['avg_exhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, True, True, hold_time])
        timer_inhale.start()

In [ ]:
# computes degree of inflation/deflation based on ECG features

# TODO: how to compute the inflation/deflation value?

def heartrate_actuation(act_id, HR, timer_interval):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    current_trend = HR.current_trends["hr_mean"]
    #current_trend = HR.current_trends["rmssd"]
    #current_trend = HR.current_trends["LF/HF ratio"]
    
    current_feature = HR.features[-1]["hr_mean"]
    #current_feature = HR.features[-1]["rmssd"]
    #current_feature = HR.features[-1]["LF/HF ratio"]
    
    if current_trend > 0:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        #osc_client.send_message("/actuator/" + act_id + "/inflate", current_trend * 1000/current_feature)
        timer_exhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_exhale.start()

    else:
        #osc_client.send_message("/actuator/inflate", current_trend * 1000/current_feature)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_inhale.start()

#### Set up configuration: IP (BITalino, R-IoT or Node-RED), port and OSC address pattern

In [ ]:
def hold_down():
    global actuation_flag
    a = 0
    return

In [ ]:
def conf_handler(unused_addr, args, *values): #Conf 

    global ecg_data, resp_data
    global actuation_flag
    global osc_client
    
    B = args[0]
    HR = args[1]   

    print("----------------- ACTUATION CHANGE ------------------------")
    if type(values[0]) == str:
        if B.routine != values[0]:
            B.routine = values[0]

        if B.routine == "None":
            actuation_flag = False
            val = 0.0
            osc_client.send_message("/actuator/1/inflate", val)
            osc_client.send_message("/actuator/2/inflate", val)

        else:
            actuation_flag = True
           
                
            print(B.routine)
            timer_act = threading.Timer(3, act_launcher, [B, resp_data, HR, ecg_data] )
            
            timer_act.start()
        
    else: 
        B.breathing_factor = np.round(values[0],1)
        print("BrF val:", B.breathing_factor)

In [ ]:
# Definitions
bitalino_ip = '192.168.0.100'
bitalino_port = 8000
bitalino_srate = 100

riot_ip = "192.168.0.100"
riot_port = 31000 
riot_srate = 200

# riot_ip = "192.168.1.40"
# riot_port = 12000 

#actuator_ip = '192.168.0.110'
actuator_ip = '192.168.0.100'
actuator_port = 32000

riot_address = "/0/raw"
bitalino_address = "/0/bitalino"


###### CONFIGURATION 
config_address = riot_address
config_ip = riot_ip
config_port = riot_port
srate = riot_srate 
C = {"config_address": config_address, "config_ip": config_ip, "config_port": config_port, "srate": srate}

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 

actuation_flag = True

ecg_data = []
resp_data = []


B = biofeatures.breathing(data = np.ones(10), buffer_length=2000, srate=srate)
HRV = biofeatures.hrv(data = np.ones(10), buffer_length=2000, srate=srate)
B.breathing_factor = 1.0
B.routine = "breathing_biofeedback" 


my_dispatcher = dispatcher.Dispatcher()
my_dispatcher.map(config_address, riot_data_handler, B, HRV)
my_dispatcher.map("/brf/", conf_handler, B, HRV)


server = osc_server.ThreadingOSCUDPServer((config_ip, config_port), my_dispatcher)
print (C)
print("Serving on {}".format(server.server_address))

# Warmup
warmup_t = 10
timer_warmup = threading.Timer(warmup_t, warmup, [B,resp_data,HRV,ecg_data])
timer_warmup.start()

try: 
    server.serve_forever()
except KeyboardInterrupt:
    B.update_data_flag = False
    HRV.update_data_flag = False
    actuation_flag = False
    server.server_close()
    osc_client.send_message("/actuator/1/inflate", 0.0)
    osc_client.send_message("/actuator/2/inflate", 0.0)
except:
    raise

## Launch on 2 devices

In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import time

# Use actuator IP
# Local Processing IP 192.168.0.100 or localhost 127.0.0.1, Arduino IP
actuator_ip = '127.0.0.1'
actuator_port = 32000
osc_client = SimpleUDPClient(actuator_ip, actuator_port) 


In [ ]:
val = 0.0
osc_client.send_message("/actuator/1/inflate", val)
osc_client.send_message("/actuator/2/inflate", val)


In [ ]:
val = 100.0
osc_client.send_message("/actuator/1/inflate", val)
osc_client.send_message("/actuator/2/inflate", val)


In [ ]:
deflate("1", 1)

In [ ]:
warmup_inflate("1", 3)

In [ ]:
warmup_inflate("2", 3)

In [ ]:
actuation_flag = True

pulsating("1", True)

In [ ]:
actuation_flag = False

In [ ]:
osc_client.send_message("/actuator/1/inflate", 0.0)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(np.linspace(0,len(resp_data)/200., len(resp_data) ), resp_data)